# Tema 6: Redes Neuronales Convolucionales (CNNs)

⚠️ RECORDATORIOS
* trabajar con GPU
* Instalar PyTorch Lightning
* Montar tu drive para guardar los archivos

In [ ]:
!pip install pytorch-lightning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import Lambda
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from torchmetrics import Accuracy, F1Score
from pytorch_lightning import Trainer
import datetime


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## DataModule

Tenemos que crear dos funciones transform:

1. train_transform:
  * Cambie el tamaño de las muestras a 150x150
  * Aplique RandAugment: número de opciones = 2, magnitud = 4
  * Pasar a tensor
  * Normalizar: transforms.Normalize((0.5,), (0.5,))

2. val_test_transform:
  * Cambie el tamaño de las muestras a 150x150
  * Pasar a tensor
  * Normalizar: transforms.Normalize((0.5,), (0.5,))




In [ ]:
class FMNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size


        self.train_transform = transforms.Compose([

                    ### CÓDIGO ###
        ])
        self.val_test_transform = transforms.Compose([

                    ### CÓDIGO ###

        ])

    def prepare_data(self):
        datasets.FashionMNIST(root="data", train=True, download=True)
        datasets.FashionMNIST(root="data", train=False, download=True)

    def setup(self, stage=None):
        if stage in (None, "fit"):
            mnist_full = datasets.FashionMNIST(root="data", train=True, transform=self.val_test_transform)
            self.train_dataset, self.val_dataset = random_split(
                mnist_full,
                [55000, 5000],
                generator=torch.Generator().manual_seed(42) ## Permite que el experimento sea reproducible
            )

            # Aplicar el transfor de train al dataset correspondiente.
            self.train_dataset.dataset.transform = self.train_transform

        if stage == "test" or stage is None:
            self.test_dataset = datasets.FashionMNIST(root="data", train=False, transform=self.val_test_transform)

    def train_dataloader(self):

                    ### CÓDIGO ###

        return

    def val_dataloader(self):

                    ### CÓDIGO ###

        return


    def test_dataloader(self):

                    ### CÓDIGO ###

        return


## Crea un CNN


Esta arquitectura debe estar compuesta por 4 capas convolucionales y sus respectivas capas BatchNormalization, funciones de activación y capas Pooling.


In [ ]:
class CNN(pl.LightningModule):
    def __init__(self, lr, input_channels, num_classes):
        super(CNN, self).__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.input_channels = input_channels
        self.num_classes = num_classes

        ### CÓDIGO ###


        self.train_accuracy = Accuracy(task='multiclass', num_classes=self.num_classes)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=self.num_classes)
        self.test_accuracy = Accuracy(task='multiclass', num_classes=self.num_classes)

    def forward(self, x):

                    ### CÓDIGO ###

        return x

    def training_step(self, batch, batch_idx):
                            ### CÓDIGO ###
        self.log_dict({
            'train_loss': loss_tr,
            'train_acc': acc_tr
            },
                      prog_bar=True,
                      on_epoch=True, on_step=False)
        return loss_tr

    def validation_step(self, batch, batch_idx):
                            ### CÓDIGO ###
        self.log_dict({
            'val_loss': loss_val,
            'val_acc': acc_val
            },
                      prog_bar=True,
                      on_epoch=True, on_step=False)

        return loss_val

    def test_step(self, batch, batch_idx):
                            ### CÓDIGO ###
        self.log_dict({
            'test_loss': loss_te,
            'test_acc': acc_te
            },
                      prog_bar=True,
                      on_epoch=True, on_step=False)

        return loss_te


    def configure_optimizers(self):
        optimizer =                     ### CÓDIGO ###
        return optimizer

In [ ]:
path =                     ### CÓDIGO ###
logger = CSVLogger(save_dir=path, name="metrics")

In [ ]:
model = ### CÓDIGO ###

In [ ]:
data_module = ### CÓDIGO ###

### Crea los siguientes Callbacks:

1. EarlyStopping:
  * Que utilice el loss de validación
  * Paciencia = 3
  * min_delta = 0.001
2. Model_checkpoint:
  * Que utilice el loss de validación
  * Guarde solo el mejor modelo

In [ ]:
early_stopping_callback = pl.callbacks.EarlyStopping(


        ### CÓDIGO ###

)
model_checkpoint_callback = pl.callbacks.ModelCheckpoint(


        ### CÓDIGO ###

)
callbacks = [early_stopping_callback, model_checkpoint_callback]

### Crear el Trainer

Incluyendo:

* Máximo de épocas = 10
* El logger, que se actualice cada época
* Los callbacks

In [ ]:
trainer = Trainer(

    ### CÓDIGO ###

)

## Entrenar el modelo

In [ ]:
trainer.fit(model, datamodule=data_module)

## Cargar el modelo y calcular las métricas para test

In [ ]:
path = "/content/drive/My Drive/Trabajo/Docencia/curso_24_25/titulo_propio/dl/tema6/results/"
model_file = "best_model_2024-12-03 15:04:30.975495.ckpt"
checkpoint_path = path + model_file
loaded_model = CNNModel.load_from_checkpoint(checkpoint_path)

## Inferencia

Mostrar 100 muestras mediante una visualización de 10 imágenes x 10 imágenes. En el título incluir la etiqueta real y la predicción

In [ ]:
def plot_predictions(model, datamodule, num_samples=100):

    rows, cols = 10, 10
    fig, axs = plt.subplots(rows, cols, figsize=(20, 20))
    axs = axs.flatten()

      ### CÓDIGO ###

    model.eval()
    model.to("cuda")


    with torch.no_grad():
        ### CÓDIGO ###


    for i in range(num_samples):
        ### CÓDIGO ###

    plt.tight_layout()
    plt.show()

In [ ]:
plot_predictions(loaded_model, data_module)